## Generate Legal Bench Classification tags using della inference API

[LegalBench](https://hazyresearch.stanford.edu/legalbench/tasks/oral_argument_question_purpose.html) has a task called `oral_argument_question_purpose` which classifies questions asked by Supreme Court justices at oral argument into seven categories:

1. Background - questions seeking factual or procedural information that is missing or not clear in the briefing
1. Clarification - questions seeking to get an advocate to clarify her position or the scope of the rule being advocated for
1. Implications - questions about the limits of a rule or its implications for future cases
1. Support - questions offering support for the advocate’s position
1. Criticism - questions criticizing an advocate’s position
1. Communicate - question designed primarily to communicate with other justices
1. Humor - questions designed to interject humor into the argument and relieve tension

This notebook uses the Llama-3.1-70B to generate classification judgments for these questions into one of these. The model is exposed through an API on della using the instructions outlined in the [`della-inference` repository](https://github.com/benediktstroebl/della-inference)

To run this notebook on della, first you will need to run the following command to set up SSH port forwarding on the della node you're on, replacing `nnadeem` with your username and `della-l05g6` with the name of the gpu node where you set up the inference API model:

```
ssh -N -L localhost:12257:della-l03g13:8000 nnadeem@della-l03g13
```

To test that the endpoint is running/SSH port forwarding is set up correctly, try the following curl command in a terminal window on the della node you're running notebook in:

```
curl http://localhost:12257/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer token-abc123" \
  -d '{
    "model": "meta-llama/Meta-Llama-3.1-70B-Instruct",
    "messages": [
      {"role": "system", "content": "Respond friendly to the user."},
      {"role": "user", "content": "Hello World!"}
    ]
  }'
```

In [1]:
import requests
import json
import pandas as pd

In [2]:
url = "http://localhost:12257/v1/chat/completions"

headers={
    "Content-Type": "application/json",
    "Authorization": "token-abc123"
}

model_name = 'meta-llama/Meta-Llama-3.1-70B-Instruct'

In [3]:
def get_legalbench_classification_prompt(opening_statement, question):
    system_prompt = """You are an expert assistant trained to classify the purpose of questions asked by judges during oral arguments. Your task is to identify the primary purpose of a given question based on the advocate's opening statement and the text of the question itself.

        ### Instructions:
        Judge's questions at oral arguments typically fall into one of the following categories:
        - 'Background': Seeks factual or procedural information missing or unclear in the briefs.
        - 'Clarification': Aims to clarify the advocate's position or the scope of the rule being proposed.
        - 'Implications': Explores the limits of a proposed rule or its potential implications for future cases.
        - 'Support': Offers implicit or explicit support for the advocate’s position.
        - 'Criticism': Challenges or criticizes the advocate's position.
        - 'Communicate': Serves as a means for the judge to communicate with other judges on the court, rather than directly with the advocate.
        - 'Humor': Introduces humor to ease tension during the proceedings.
        Your output should classify the judge's question into one of these categories and explain your reasoning.


        ### Output format:
        Your response must follow this JSON format:
        {
        "classification": "<Category Name>",
        "reasoning": "<A brief explanation for the classification>"
        }


        ### Example:
        Opening Statement: "Your Honors, we contend that the statute should be interpreted in light of its original intent, which clearly establishes a narrow scope of application to avoid overreach."
        Question: "If we accept your interpretation, how would it apply to cases involving modern technologies not contemplated when the statute was written?"
                
        ### Response:
        {
        "classification": "Implications",
        "reasoning": "The question explores the potential limits and future consequences of the advocate’s proposed rule in unanticipated scenarios."
        }


    """
    
    user_prompt = f"""### Your Task:
        Opening Statement: {opening_statement}
        Question: {question}

        ### Response:
    """

    messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": user_prompt}
        ]
    return messages

In [34]:
def get_model_response(messages):

    payload = {
        "model": model_name,
        "messages": messages
    }

    response = requests.post(url, data=json.dumps(payload), headers=headers)
    return response

def parse_response(response):
    decoded = response.content.decode('utf-8')
    response_data = json.loads(decoded)
    content = response_data['choices'][0]['message']['content']

    # try:
    #     tags = json.dumps(content)
    # except Exception as e:
    #     print(f"Unable to jsonify response, saving string itself. ERROR: {e}")
    #     tags = str(content)
    # return tags
    
    return content

In [35]:
def classify_questions_legalbench(opening_statement, question):
    messages = get_legalbench_classification_prompt(opening_statement, question)
    response = get_model_response(messages)
    tags = parse_response(response)
    return tags

#### Get labels for 2024 'coherent' questions

In [36]:
input_fp = '../datasets/2024_all_questions_coherence_labeled_Meta-Llama-3.1-70B-Instruct.csv'
df = pd.read_csv(input_fp)
df.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label
0,2024.23-621-t01,petitioner,Clarence Thomas,You --can a consent decree or a default judgm...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
1,2024.23-621-t01,petitioner,Clarence Thomas,But I thought your argument hinged on a court...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent
2,2024.23-621-t01,petitioner,"John G. Roberts, Jr.",What do you do with the formulation by your f...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
3,2024.23-621-t01,petitioner,Elena Kagan,"Well, it's -- it's true that it's only a lik...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent
4,2024.23-621-t01,petitioner,Ketanji Brown Jackson,But it's not that determination that's making...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent


In [37]:
df_coherent = df[df['label'] == 'coherent']
df_coherent = df_coherent.copy()

In [38]:
# GENERATE FOR ALL
df_coherent['legalbench_classification_raw'] = df_coherent.apply(
    lambda row: classify_questions_legalbench(row['opening_statement'], row['question_text']), axis=1
)
df_coherent.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label,legalbench_classification_raw
1,2024.23-621-t01,petitioner,Clarence Thomas,But I thought your argument hinged on a court...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n""classification"": ""Clarification"",\n""reason..."
3,2024.23-621-t01,petitioner,Elena Kagan,"Well, it's -- it's true that it's only a lik...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n""classification"": ""Criticism"",\n""reasoning""..."
4,2024.23-621-t01,petitioner,Ketanji Brown Jackson,But it's not that determination that's making...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n""classification"": ""Criticism"",\n""reasoning""..."
5,2024.23-621-t01,petitioner,Ketanji Brown Jackson,When you think about the difference between m...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""classification"": ""Clarification"",\n ""re..."
7,2024.23-621-t01,petitioner,Ketanji Brown Jackson,But didn't Sole open -- leave open that --th...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n""classification"": ""Criticism"",\n""reasoning""..."


Save File:

In [42]:
out_fp = '../datasets/2024_coherent_legalbench_classification.csv'
df_coherent.to_csv(out_fp, index=False)

### Run on incoherent

In [45]:
input_fp = '../datasets/2024_all_questions_coherence_labeled_Meta-Llama-3.1-70B-Instruct.csv'
df = pd.read_csv(input_fp)
# df.head()

In [46]:
df_incoherent = df[df['label'] == 'incoherent']
df_incoherent = df_incoherent.copy()
df_incoherent.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label
0,2024.23-621-t01,petitioner,Clarence Thomas,You --can a consent decree or a default judgm...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
2,2024.23-621-t01,petitioner,"John G. Roberts, Jr.",What do you do with the formulation by your f...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
6,2024.23-621-t01,petitioner,Ketanji Brown Jackson,What about the Chief Justice's example? In t...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
8,2024.23-621-t01,petitioner,Elena Kagan,"And -- and -- and couldn't a state do that, y...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
11,2024.23-621-t01,petitioner,"Samuel A. Alito, Jr.",If there is very strong evidence that the gov...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent


In [47]:
# GENERATE FOR ALL
df_incoherent['legalbench_classification_raw'] = df_incoherent.apply(
    lambda row: classify_questions_legalbench(row['opening_statement'], row['question_text']), axis=1
)
df_incoherent.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label,legalbench_classification_raw
0,2024.23-621-t01,petitioner,Clarence Thomas,You --can a consent decree or a default judgm...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n""classification"": ""Background"",\n""reasoning..."
2,2024.23-621-t01,petitioner,"John G. Roberts, Jr.",What do you do with the formulation by your f...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n""classification"": ""Criticism"",\n""reasoning""..."
6,2024.23-621-t01,petitioner,Ketanji Brown Jackson,What about the Chief Justice's example? In t...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n ""classification"": ""Criticism"",\n ""reason..."
8,2024.23-621-t01,petitioner,Elena Kagan,"And -- and -- and couldn't a state do that, y...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n""classification"": ""Implications"",\n""reasoni..."
11,2024.23-621-t01,petitioner,"Samuel A. Alito, Jr.",If there is very strong evidence that the gov...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n""classification"": ""Implications"",\n""reasoni..."


Save file:

In [48]:
out_fp = '../datasets/2024_incoherent_legalbench_classification.csv'
df_coherent.to_csv(out_fp, index=False)

### Test on sample

#### Test full workflow

In [32]:
# # test on subsample (full workflow
df_new = df_coherent.head(2).copy()
df_new['legalbench_classification_raw'] = df_new.apply(
    lambda row: classify_questions_legalbench(row['opening_statement'], row['question_text']), axis=1
)
df_new

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label,legalbench_classification_raw
1,2024.23-621-t01,petitioner,Clarence Thomas,But I thought your argument hinged on a court...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n""classification"": ""Clarification"",\n""reason..."
3,2024.23-621-t01,petitioner,Elena Kagan,"Well, it's -- it's true that it's only a lik...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n""classification"": ""Criticism"",\n""reasoning""..."


In [33]:
sample_resp = json.loads(df_new['legalbench_classification_raw'][1])
sample_resp

{'classification': 'Clarification',
 'reasoning': "The judge is seeking to understand the advocate's position, specifically whether their argument relies on a court ruling in favor of the prevailing party on the merits, which suggests the judge may be unclear about the advocate's stance or the scope of their proposed rule."}

#### Test each step separately to confirm jsonification

In [ ]:
sample_opening = df_new.iloc[0]['opening_statement']
sample_question = df_new.iloc[0]['question_text']
messages = get_legalbench_classification_prompt(sample_opening, sample_question)
response = get_model_response(messages)

In [25]:
sample_opening = df_new.iloc[0]['opening_statement']
sample_question = df_new.iloc[0]['question_text']
messages = get_legalbench_classification_prompt(sample_opening, sample_question)
response = get_model_response(messages)
response

<Response [200]>

In [28]:
decoded = response.content.decode('utf-8')
response_data = json.loads(decoded)
content = response_data['choices'][0]['message']['content']
content

'{\n"classification": "Clarification",\n"reasoning": "The judge is seeking to understand the advocate\'s position, specifically the relationship between a court ruling on the merits and the prevailing party, to ensure they accurately grasp the advocate\'s argument."\n}'

In [29]:
content_json_obj = json.loads(content)
content_json_obj

{'classification': 'Clarification',
 'reasoning': "The judge is seeking to understand the advocate's position, specifically the relationship between a court ruling on the merits and the prevailing party, to ensure they accurately grasp the advocate's argument."}